# Cryptocompare API Exercise

Going through the API notebook using the [Cryptocompare.com](https://www.cryptocompare.com/) API as it is relevant to the question I want to answer in my EDA project.

In [1]:
import requests

In [6]:
# Make a request to get historical Bitcoin data
response = requests.get('https://min-api.cryptocompare.com/data/histoday')

print(response.status_code)

200


In [26]:
params = {'fsym': 'BTC', 'tsym': 'USD', 'limit': 100}

response = requests.get('https://min-api.cryptocompare.com/data/histoday', params=params)

print(response.content)

b'{"Response":"Success","Type":100,"Aggregated":false,"Data":[{"time":1512432000,"close":11667.13,"high":11901.87,"low":11486.13,"open":11624.37,"volumefrom":89687.21,"volumeto":1048839303.8},{"time":1512518400,"close":13749.57,"high":13843.2,"low":11661.76,"open":11667.13,"volumefrom":191576.66,"volumeto":2437037791.81},{"time":1512604800,"close":16850.31,"high":16879.26,"low":13401.61,"open":13750.09,"volumefrom":297108.66,"volumeto":4510225316.19},{"time":1512691200,"close":16047.61,"high":17294.85,"low":13906.1,"open":16867.98,"volumefrom":286762.02,"volumeto":4546014731.58},{"time":1512777600,"close":14843.42,"high":16313.18,"low":13151.47,"open":16048.18,"volumefrom":181979.81,"volumeto":2699876215.33},{"time":1512864000,"close":15059.6,"high":15783.2,"low":13031,"open":14839.98,"volumefrom":201620.09,"volumeto":2904037859.64},{"time":1512950400,"close":16732.47,"high":17399.18,"low":15024.56,"open":15060.45,"volumefrom":159724.56,"volumeto":2634267594.52},{"time":1513036800,"clo

In [11]:
import json

In [15]:
data = response.json()['Data']
print(data[0])

{'time': 1519344000, 'close': 10175.51, 'high': 10420.67, 'low': 9604, 'open': 9847.96, 'volumefrom': 139373.76, 'volumeto': 1399448203.16}


In [16]:
import pandas as pd
import datetime

In [17]:
df = pd.DataFrame(data)
df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]

In [18]:
df.head()

,close,high,low,open,time,volumefrom,volumeto,timestamp
0,10175.51,10420.67,9604.00,9847.96,1519344000,139373.76,1.399448e+09,2018-02-22 18:00:00
1,9705.73,10528.00,9396.54,10175.51,1519430400,125269.68,1.244484e+09,2018-02-23 18:00:00
2,9610.11,9873.79,9329.44,9705.73,1519516800,82430.45,7.930932e+08,2018-02-24 18:00:00
3,10326.50,10457.51,9411.82,9610.11,1519603200,117416.47,1.181729e+09,2018-02-25 18:00:00
4,10594.76,10879.38,10154.24,10326.50,1519689600,99772.39,1.060841e+09,2018-02-26 18:00:00


In [19]:
df = df[['open', 'high', 'low', 'close', 'volumeto', 'timestamp']]
df.head()

,open,high,low,close,volumeto,timestamp
0,9847.96,10420.67,9604.00,10175.51,1.399448e+09,2018-02-22 18:00:00
1,10175.51,10528.00,9396.54,9705.73,1.244484e+09,2018-02-23 18:00:00
2,9705.73,9873.79,9329.44,9610.11,7.930932e+08,2018-02-24 18:00:00
3,9610.11,10457.51,9411.82,10326.50,1.181729e+09,2018-02-25 18:00:00
4,10326.50,10879.38,10154.24,10594.76,1.060841e+09,2018-02-26 18:00:00


In [27]:
# Create list of coins that we want databases for
coin_list = ['BTC', 'ETH', 'XRP', 'BCH', 'LTC', 'ADA', 'NEO', 'XLM', 'EOS', 'XMR']

# Create a function to generate desired dataframe for given coin name
def generate_historical_dataframe(sym, curr='USD', limit=100):
    url = 'https://min-api.cryptocompare.com/data/histoday'
    params = {'fsym': sym, 'tsym': curr, 'limit': limit}
    
    response = requests.get(url, params=params)
    data = response.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    df = df[['open', 'high', 'low', 'close', 'volumeto', 'timestamp']]
    
    return df

In [28]:
import sqlite3

# Write function to create a database for all coins in coinlist
def create_database(df, table_name, sqlite_file):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()

    df.to_sql(table_name, conn, if_exists='replace')
    
    conn.commit()
    conn.close()

In [31]:
create_database(df, "TEST1", 'test_db.sqlite')

In [32]:
# Name of the database file
sqlite_file = 'cryptocompare_db.sqlite'

# Loop through coin list, get historical data, create new database
for coin in coin_list:
    coin_df = generate_historical_dataframe(coin)
    create_database(coin_df, coin, sqlite_file)